In [1]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [2]:
@inbounds U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
@inbounds σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
@inbounds g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )

@inbounds function TM(u, p, t)
    
    U_ = U(u[3], p)
    
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return SVector(du1, du2, du3)
end
@inbounds function jacob_TM_(u, p, t)
    
    U(y, p, exp50) = p[8] + p[9] / ( 1.0 + exp50 )
    U_y(y, p, exp50) = (50.0 * p[9] * exp50) / (1.0 + exp50)^2
    g(E, x, y, p, U_) = exp((p[5]  * U_ * x * E + p[11]) / p[1])
    σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
    exp50 = exp(-50.0 * (u[3] - p[7]))
    
    U_ = U(u[3], p, exp50)
    Uy = U_y(u[3], p, exp50)
    g_ = g(u[1], u[2], u[3], p, U_)
    σ_deri = σ_der(u[2], p)
    
    g_plus = 1.0 + g_
    g_mult = g_ * U_
    g_plus_mult = p[2] * (g_plus)
    u1p5 = p[5] * u[1]
    Uyu2 = Uy * u[2]
    
    E_E = (-1.0 + ((J * u[2] * g_mult)) / (g_plus) ) / p[2]
    E_x = (u1p5 * g_mult) / (g_plus_mult)
    E_y = (u1p5 * Uyu2 * g_) / (g_plus_mult)
    
    x_E = -U_ * u[2]
    x_x = -1.0 / p[3] - U_ * u[1]
    x_y = -Uyu2 * u[1]
    
    y_x = 20.0 * p[10] * σ_deri / (1.0 + σ_deri)^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end;

In [3]:
Pxindex = 11; Pyindex = 8 # Индекс параметра по оси х и оси у
Pxlabel = "I0"; Pylabel = "U0"; # Обозначение параметров. Нужно для вывода.

In [27]:
# Диапазон для параметров
Pxrange = range( -0.5, -2.0,  length = 200 )
Pyrange = range( 0.22,  0.55, length = 200 )

const Px_start = Pxrange[1]
const Py_start = Pyrange[1];

In [5]:
D = 3
# Массивы для хранения Спектра и начальных условий
Λs = zeros(length(Pxrange), length(Pyrange), D)
u0s = zeros(length(Pxrange), length(Pyrange), D);

In [6]:
# Имена файлов, в которые сохранится матрица спектра и начальных условий
file_name_LS = "LSs_TEST.jld"
file_name_u0s = "u0s_TEST.jld"

"u0s_TEST.jld"

In [25]:
function init_ds_Px(p, Px, Pxindex, u0)
    p[Pxindex] = Px; p[Pyindex] = Py_start
    ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
end
function init_ds_Py(p, Py, Pyindex, u0)
    p[Pyindex] = Py; p[Pxindex] = Px_start
    ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
    return ds
end
function init_ds(p, Pxindex, Pyindex, Px, Py, u0)
    p[Pyindex] = Py; p[Pxindex] = Px
    ds = CoupledODEs(TM, u0, p,  diffeq = integ_set)
    return ds
end
function goto_attractor(ds, t, tt)
    tr, _ = trajectory(ds, t; Ttr = tt, Δt = tstep)
    u0 = tr[end]
    return u0
end
function spectrum(ds)
    tan_ = TangentDynamicalSystem(ds, J = jacob_TM_)
    ΛΛ = lyapunovspectrum(ds, t)
    return ΛΛ
end

spectrum (generic function with 1 method)

In [20]:
function save_output_Px(idx_Px, ΛΛ, u0)
    Λs[idx_Px, 1, :] = ΛΛ
    u0s[idx_Px, 1, :] = u0
end
function save_output_Py(idx_Py, ΛΛ, u0)
    Λs[1, idx_Py, :] = ΛΛ
    u0s[1, idx_Py, :] = u0
end
function save_output(idx_Px, idx_Py, ΛΛ, u0)
    Λs[idx_Px, idx_Py, :] = ΛΛ
    u0s[idx_Px, idx_Py, :] = u0
end
function save_tofile()
    save(file_name_LS, "data", Λs)
    save(file_name_u0s, "data", u0s)
end

save_tofile (generic function with 1 method)

In [21]:
function outputPx(idxPx, Px, u0)
    println("Индекс $Pxlabel: $idxPx; $Pxlabel: $Px");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function outputPy(idxPx, Py, u0)
    println("Индекс $Pylabel: $idxPx; $Pylabel: $Py");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_Px, idx_Py, Px, Py, u0)
    println("Индекс $Pxlabel: $idx_Px; $Pxlabel: $Px");flush(stdout)
    println("Индекс $Pylabel: $idx_Py; $Pylabel: $Py");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function outputIter(ΛΛ, u0)
    println("Спектр: $ΛΛ");flush(stdout)
    println("Последняя точка: $u0");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [11]:
"""
t -- Время в течении которого вычисляется спектр
tt -- Время которое отбрасывается
tstep -- Шаг по времени для интегратора
time_attract -- Время в течении которого считается траектория

integ_set -- Tuple для настроек интегратора
"""
t = 1000; tt = 500; tstep = 0.001
time_attract = 100.0

integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [9]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.22;  const ΔU0 = 0.305; I0 = -0.5

p = [α, τ, τD, τy, J, xthr, ythr, 0, ΔU0, β, 0]

u0 = [8.39353, 0.73803, 0.429055]

3-element Vector{Float64}:
 8.39353
 0.73803
 0.429055

In [ ]:
# INHERITANCE RIGHT-LEFT

for (idx_Px, Px) in enumerate(Pxrange)
    
    if idx_Px == 1 # Проверка на первую итерацию
        global u0_lc = u0
    end
    
    outputPx(idx_Px ,Px, u0_lc)
    
    ds = init_ds_Px(p, Px, Pxindex, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract, tt) # Выход на аттрактор
    ds = init_ds_Px(p, Px, Pxindex, u0_lc)
    ΛΛ = spectrum(ds) # Вычисление спектра
    
    outputIter(ΛΛ, u0_lc)
    
    save_output_Px(idx_Px, ΛΛ, u0_lc) # Сохранение значений
    
    separate()
    #save_tofile()
end

In [ ]:
for (idx_Px, Px) in enumerate(Pxrange)
    for (idx_Py, Py) in enumerate(Pyrange)
    
        if idx_Py == 1 # Если использовать while условие не нужно!
            continue
        end
        
        u0_lc = u0s[idx_Px, idx_Py- 1, :]
        
        output(idx_Px, idx_Py, Px, Py, u0_lc)
        
        ds = init_ds(p, Pxindex, Pyindex, Px, Py, u0)
        u0_lc = goto_attractor(ds, time_attract, tt) # Выход на аттрактор
        ds = init_ds(p, Pxindex, Pyindex, Px, Py, u0)
        ΛΛ = spectrum(ds) # Вычисление спектра
        
        outputIter(ΛΛ, u0_lc)
        
        save_output(idx_Px, idx_Py, ΛΛ, u0_lc)
        
        separate()
    end
    # save_tofile()
end